<a href="https://colab.research.google.com/github/UetaKento/Aizu_NLP/blob/main/IRNLP2019_Ex03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 3. Processing Raw Text (3.1, 2-6, 9)

For International Students: goto http://www.nltk.org/book/ch03.html  
Almost corresponded about:
 - Lesson 1: 3.1
 - Lesson 2: 3.3
 - Lesson 3: 3.4
 - Lesson 4: 3.5
 - Lesson 5: 3.6
 - Lesson 6: 3.9
 
(★ Assignment Remark): Please read carefully about 3.1 and 3.3.

この演習では、Webや平文のデータをどうやって読み込み、言語資源として利用するかを学ぶ。  
また、言語リソースとして扱うために必要な処理として、

 - トークン化
 - ステミング (Stemming)/ 見出し語化 (Lemmatizing)
 - セグメンテーション

等のことをNLTKでどのように行うかを学ぶ。

※ 以下の環境では、特にしていなければ以下のimport文から開始された対話セッション, あるいはコードであると仮定して進めることとする。

In [ ]:
#from __future__ import division  # Python 2 users only
import nltk, re, pprint
nltk.download('all')

## Lesson 1. Accessing Text from the Web and from Disk (3.1.)

### 1.1. Electronic Book

電子書籍 (Project Gutenberg) からの参照をWeb経由で行うために。
- http://www.gutenberg.org/catalog/
- 25,000件の無料オンライン書籍のカタログ
- 50以上の言語
- text[2554] : an English translation of Crime and Punishment (罪と罰, ドスドエフスキー）

なお、Project Gutenbergのテキストを読み込もうとすると、過去のNLTK Bookの中にあるWebページではアクセスできなくなっている（2554-0.txt）ことから、URLを見直す必要がある。  
また、Python 3から、UrlLibは、urlopen()メソッドがrequest内に配置されているため、そのように変更する。

In [ ]:
from nltk import word_tokenize
from urllib import request
url = "http://www.gutenberg.org/files/2554/2554-0.txt"
response = request.urlopen(url)
raw = response.read().decode('utf8')
print('type:', type(raw), ' length:', len(raw))
print(raw[:200])

type: <class 'str'>  length: 1176812
﻿The Project Gutenberg eBook of Crime and Punishment, by Fyodor Dostoevsky

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with a


言語処理を行うために、文字列を単語と句読点に分解する。この処理をトークン化 (Tokenization)と呼ぶ。
- 文字列を単語と句読点に分割。
- 空白や改行、空白行を除去

In [ ]:
tokens = word_tokenize(raw)
print('type:', type(tokens), ' length:', len(tokens))
print(tokens[:20])

type: <class 'list'>  length: 257712
['\ufeffThe', 'Project', 'Gutenberg', 'eBook', 'of', 'Crime', 'and', 'Punishment', ',', 'by', 'Fyodor', 'Dostoevsky', 'This', 'eBook', 'is', 'for', 'the', 'use', 'of', 'anyone']


トークン化された情報を利用すれば、Ex. 1で紹介したSlicing等の言語処理を行うことが出来る。
- nltk.Textを用いてトークン化された情報をリスト化する

In [ ]:
text = nltk.text.Text(tokens)
print('type:', type(text))
print(text[1024:1062])
print(text.collocations())

type: <class 'nltk.text.Text'>
['wisdom', '...', 'that', 'wisdom', 'of', 'the', 'heart', 'which', 'we', 'seek', 'that', 'we', 'may', 'learn', 'from', 'it', 'how', 'to', 'live', '.', 'All', 'his', 'other', 'gifts', 'came', 'to', 'him', 'from', 'nature', ',', 'this', 'he', 'won', 'for', 'himself', 'and', 'through', 'it']
Katerina Ivanovna; Pyotr Petrovitch; Pulcheria Alexandrovna; Avdotya
Romanovna; Rodion Romanovitch; Marfa Petrovna; Sofya Semyonovna; old
woman; Project Gutenberg-tm; Porfiry Petrovitch; Amalia Ivanovna;
great deal; young man; Nikodim Fomitch; Project Gutenberg; Ilya
Petrovitch; Andrey Semyonovitch; Hay Market; Dmitri Prokofitch; Good
heavens
None


※ collocations() is buggy. Please use collocation_list() to get word collocation list in Python 3.  
collocations()にはバグが含まれている可能性がありますので、使用する場合はcollocation_list()を利用してください。

語彙のリスト化が行えると、その文書から、単語ごとの検索を行うことが出来る。  
（fund, rfindを利用する)

In [ ]:
print('position of "PART IV":', raw.find("PART IV"))
print('reverse position of "End of Project Gutenberg’s Crime":', raw.rfind("End of Project Gutenberg’s Crime"))
a = raw.find("PART IV")
raw2 = raw[a:]
print('position of "PART IV" (cutted out):', raw2.find("PART IV"))

position of "PART IV": 596072
reverse position of "End of Project Gutenberg’s Crime": -1
position of "PART IV" (cutted out): 0


### 1.2. Processing HTML

(★ Assignment Remark) Web上のドキュメントの殆どは、HTML形式で保存されているので、そうしたWeb上のテキストを収集し保存しておけば、一度アクセスするだけでその後は何度も閲覧できる。（キャッシュと呼んでいる。）  

HTMLを扱う場合: urlopenを利用してデータを取得し、保存しておく。  
対象とするドキュメント: BBC "Blondes to die out in 200 years" (ブロンドは200年後に死に絶える) = 都市伝説

In [ ]:
url = "http://news.bbc.co.uk/2/hi/health/2284783.stm"
html = request.urlopen(url).read().decode('utf8')
html[:200]

'<!doctype html public "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">\r\n<html>\r\n<head>\r\n<title>BBC NEWS | Health | Blondes \'to die out in 200 years\'</title>\r\n<meta '

HTMLには『タグ』が含まれている (大抵、山かっこ '<', '>' で囲まれている）。  
この『タグ』を除去して、テキストを抽出する必要がある（Parsingと呼んだりする)。
HTMLからテキストの抽出では、PythonではBeautifulSoupを利用してタグの除去を行う。

※ BeautifulSoupが入ってないかもしれないので、利用できない場合は飛ばしてください。

インストール方法 (各自のPC上で）
 - pip install -U bs4


In [ ]:
from bs4 import BeautifulSoup
raw = BeautifulSoup(html).get_text()
tokens = word_tokenize(raw)
print(tokens[:100])

['BBC', 'NEWS', '|', 'Health', '|', 'Blondes', "'to", 'die', 'out', 'in', '200', "years'", 'NEWS', 'SPORT', 'WEATHER', 'WORLD', 'SERVICE', 'A-Z', 'INDEX', 'SEARCH', 'You', 'are', 'in', ':', 'Health', 'News', 'Front', 'Page', 'Africa', 'Americas', 'Asia-Pacific', 'Europe', 'Middle', 'East', 'South', 'Asia', 'UK', 'Business', 'Entertainment', 'Science/Nature', 'Technology', 'Health', 'Medical', 'notes', '--', '--', '--', '--', '--', '--', '-', 'Talking', 'Point', '--', '--', '--', '--', '--', '--', '-', 'Country', 'Profiles', 'In', 'Depth', '--', '--', '--', '--', '--', '--', '-', 'Programmes', '--', '--', '--', '--', '--', '--', '-', 'SERVICES', 'Daily', 'E-mail', 'News', 'Ticker', 'Mobile/PDAs', '--', '--', '--', '--', '--', '--', '-', 'Text', 'Only', 'Feedback', 'Help', 'EDITIONS', 'Change', 'to', 'UK']


内容を見ると、タグを除去しただけでは、ナビゲーション等に使われている文言がそのまま残ってしまうので、本文の部分を抽出するのには、一定の試行錯誤が必要となる場合が多い。

In [ ]:
tokens = tokens[110:390]
text = nltk.Text(tokens)
text.concordance('gene')

Displaying 5 of 5 matches:
hey say too few people now carry the gene for blondes to last beyond the next 
blonde hair is caused by a recessive gene . In order for a child to have blond
 have blonde hair , it must have the gene on both sides of the family in the g
ere is a disadvantage of having that gene or by chance . They do n't disappear
des would disappear is if having the gene was a disadvantage and I do not thin


### 1.3. Processing from the result of search engine
検索エンジン
 - 検索エンジンも一つのテキストの巨大なコーパスとみることができる。
 - 特に、興味のある言語パターンを見つける可能性が高い
 - 検索パターンの許容範囲制限があったり、一貫性のない結果によるバイアスが生じる。


### 1.4. RSS Feed
RSS Feedを用いる
- Universal Feed Parserを使うと、RSS Feedもそのまま読める

※ feedparserも入ってないかもしれない。
- pip install -U feedparser

以下の例は、その日のFeedによってタイトルが変わる。

In [ ]:
!pip install -U feedparser

     |████████████████████████████████| 81 kB 5.8 MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=f58368dee1b6c708bb175b822366ff14672ff0746a4afe6485eacb9e5f13c343
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built sgmllib3k


In [ ]:
import feedparser
llog = feedparser.parse("http://languagelog.ldc.upenn.edu/nll/?feed=atom")
print(llog['feed']['title'])
print('length:',len(llog.entries))
post = llog.entries[2]
print('title:', post.title)
content = post.content[0].value
print('content:', content[:200])
# Text book はnltkでやっていますが、既にbs4で行う形に置き換えられています。
raw = BeautifulSoup(content).get_text()
print(word_tokenize(raw)[:100])

Language Log
length: 13
title: &quot;Clear&quot; and &quot;turbid&quot; in Chinese phonology, part 4
content: <p>[This is a guest post by W. South Coblin in response to these questions which I asked him about the distinction between qing 清 ("clear") and zhuo 濁 ("muddy; turbid") in Chinese language studies:</p
['[', 'This', 'is', 'a', 'guest', 'post', 'by', 'W.', 'South', 'Coblin', 'in', 'response', 'to', 'these', 'questions', 'which', 'I', 'asked', 'him', 'about', 'the', 'distinction', 'between', 'qing', '清', '(', '``', 'clear', "''", ')', 'and', 'zhuo', '濁', '(', '``', 'muddy', ';', 'turbid', "''", ')', 'in', 'Chinese', 'language', 'studies', ':', '1.', 'when', 'and', 'how', 'it', 'arose', '2.', 'how', 'it', 'functions', 'within', 'traditional', 'Chinese', 'phonology', '3.', 'how', 'it', 'correlates', 'with', 'concepts', 'in', 'modern', 'linguistics', ']', 'What', 'you', '’', 're', 'asking', 'for', 'would', 'require', 'a', 'treatise', ',', 'or', 'maybe', 'even', 'a', 'monograph', 'on'

### 1.5. Reading Local Files

ローカルファイルを読み込むには: 
 - pythonのfile openを使う。

In [ ]:
f = open('document.txt')

そのまま利用するとエラーが発生するため、先に自分の居るディレクトリにファイルがあるかを確認する。

In [ ]:
import os
print(os.listdir('.')[:10])

['.config', 'document.txt', 'sample_data']


read()関数: そのまま、ファイル全体をメモリに読み出す。

In [ ]:
print(f.read())

Time flies like an arrow.
Fruit flies like a banana.



open()関数を用いてループしながら行ごとに読む

In [ ]:
f = open('document.txt', 'r')
for line in f:
    print(line.strip())

Time flies like an arrow.
Fruit flies like a banana.


nltk.data.find()を用いてパスを取得する。

In [ ]:
path = nltk.data.find('corpora/gutenberg/melville-moby_dick.txt')
raw = open(path, 'r').read()
print(raw[:100])

[Moby Dick by Herman Melville 1851]


ETYMOLOGY.

(Supplied by a Late Consumptive Usher to a Grammar


### 1.6. PDF/ MSWord等のファイルを用いたい場合
- 基本的にはできません。（予め、ファイルを開き、テキストとしてローカルドライブに保存する必要がある）
- PDF: pip install pdfminer3k
- Word: pip install textract

### 1.7. ユーザ入力のキャプチャ

In [ ]:
s = input("Enter some text: ")

Enter some text: aaa


In [ ]:
print("You typed", len(word_tokenize(s)), "words.")

NameError: ignored

### 1.8. NLP Pipeline

![image.png](attachment:image.png)

上図は本章で扱う処理をまとめた図である。概要として、『読み込み』、『タグ等の除去』、『本文の抽出』、『トークン化』、『語彙の正規化』を行う。  
なお、正規化には、この後に説明する、StemmingやLemmatizing等の技術が必要となる。

In [ ]:
raw = open('document.txt').read()
print('raw type:',type(raw))
tokens = word_tokenize(raw)
print('tokens type:',type(tokens))
words = [w.lower() for w in tokens]
print('words type:',type(words))
vocab = sorted(set(words))
print('vocab type:',type(vocab))
print(vocab[:10])

raw type: <class 'str'>


NameError: ignored

Wrong Case 1: appendを文字列型に利用しようとするとエラー

vocab.append('blog')
raw.append('blog')

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-33-9b21c932c920> in <module>
      1 vocab.append('blog')
----> 2 raw.append('blog')

AttributeError: 'str' object has no attribute 'append'

Wrong Case 2: 文字列にリストを接合（できない）

query = 'Who knows?'
beatles = ['john', 'paul', 'george', 'ringo']
query + beatles

```
---------------------------------------------------------------------------

TypeError                                 Traceback (most recent call last)
<ipython-input-34-67c1a27cd064> in <module>
      1 query = 'Who knows?'
      2 beatles = ['john', 'paul', 'george', 'ringo']
----> 3 query + beatles

TypeError: can only concatenate str (not "list") to str
```




- Unicode: 100万字以上のサポート。マルチバイト。

### 2.1. Extract text from encoded file (3.3.2)
ファイルからエンコードされたテキストを抽出。 (Unicode)  
latin2 エンコードに変更

In [ ]:
path = nltk.data.find('corpora/unicode_samples/polish-lat2.txt')
f = open(path, encoding='latin2')
for line in f:
    line = line.strip()
    print(line)

Pruska Biblioteka Państwowa. Jej dawne zbiory znane pod nazwą
"Berlinka" to skarb kultury i sztuki niemieckiej. Przewiezione przez
Niemców pod koniec II wojny światowej na Dolny Śląsk, zostały
odnalezione po 1945 r. na terytorium Polski. Trafiły do Biblioteki
Jagiellońskiej w Krakowie, obejmują ponad 500 tys. zabytkowych
archiwaliów, m.in. manuskrypty Goethego, Mozarta, Beethovena, Bacha.


ユニコードエスケープを付与した形での表現 (頭に b' が付く。　\\u0144等のバイナリコードが付く）

In [ ]:
f = open(path, encoding='latin2')
for line in f:
    line = line.strip()
    print(line.encode('unicode_escape'))

b'Pruska Biblioteka Pa\\u0144stwowa. Jej dawne zbiory znane pod nazw\\u0105'
b'"Berlinka" to skarb kultury i sztuki niemieckiej. Przewiezione przez'
b'Niemc\\xf3w pod koniec II wojny \\u015bwiatowej na Dolny \\u015al\\u0105sk, zosta\\u0142y'
b'odnalezione po 1945 r. na terytorium Polski. Trafi\\u0142y do Biblioteki'
b'Jagiello\\u0144skiej w Krakowie, obejmuj\\u0105 ponad 500 tys. zabytkowych'
b'archiwali\\xf3w, m.in. manuskrypty Goethego, Mozarta, Beethovena, Bacha.'


Pythonでは、通常の文字リテラルの前にuを付けることで、Unicode文字列リテラルとすることが出来る (u'a'など).   

ordを利用して文字の序列を確認する: nacuteと呼ばれる文字を表現する

In [ ]:
print(ord('ń'))
nacute = '\u0144'
print(nacute)
print(nacute.encode('utf8'))

324
ń
b'\xc5\x84'


unicodedataモジュールはUnicode文字の属性を調べる機能を提供する。

In [ ]:
import unicodedata
lines = open(path, encoding='latin2').readlines()
line = lines[2]
print(line.encode('unicode_escape'))

b'Niemc\\xf3w pod koniec II wojny \\u015bwiatowej na Dolny \\u015al\\u0105sk, zosta\\u0142y\\n'


In [ ]:
for c in line:
    if ord(c) > 127:
        print('{} U+{:04x} {}'.format(c.encode('utf8'), ord(c), unicodedata.name(c)))

b'\xc3\xb3' U+00f3 LATIN SMALL LETTER O WITH ACUTE
b'\xc5\x9b' U+015b LATIN SMALL LETTER S WITH ACUTE
b'\xc5\x9a' U+015a LATIN CAPITAL LETTER S WITH ACUTE
b'\xc4\x85' U+0105 LATIN SMALL LETTER A WITH OGONEK
b'\xc5\x82' U+0142 LATIN SMALL LETTER L WITH STROKE


Pythonの文字列メソッドとre(正規表現)おジュールがUnicode文字列をどのように扱うか？

In [ ]:
print('zosta: ', line.find('zosta\u0142y'))
line = line.lower()
print('line: ', line)
print('unicode: ', line.encode('unicode_escape'))
import re
m = re.search('\u015b\w*', line)
print('searched: ', m.group())
print(word_tokenize(line))

zosta:  54
line:  niemców pod koniec ii wojny światowej na dolny śląsk, zostały

unicode:  b'niemc\\xf3w pod koniec ii wojny \\u015bwiatowej na dolny \\u015bl\\u0105sk, zosta\\u0142y\\n'
searched:  światowej
['niemców', 'pod', 'koniec', 'ii', 'wojny', 'światowej', 'na', 'dolny', 'śląsk', ',', 'zostały']


### 2.2. Pythonでローカルエンコーディングを使用する。
![image.png](attachment:image.png)

## Lesson 3. Regular Expression to extract word pattern (3.4.)

(★ Assingment Remark) 例えば、過去形 "ed" で終わる単語を見つける等と言った作業では、正規表現などを用いてマッチングさせることによって、その単語の情報を取得することが出来る。  
凡そ、正規表現については、他の書籍も多くあることなので、是非そちらを参照されたいが、自然言語処理に必要と思われる部分を中心に例示する。

下例では、語彙リストコーパスを用いる。予め、小文字にしておく。

In [ ]:
import re
wordlist = [w for w in nltk.corpus.words.words('en') if w.islower()] # 単語を全て小文字にしておく

### 3.1. 基本的なメタキャラクタの利用

例えば、過去形"ed" で終わる単語を探すためには、('ed$') という正規表現を利用することが出来る。

In [ ]:
print([w for w in wordlist if re.search('ed$', w)][:100])

['abaissed', 'abandoned', 'abased', 'abashed', 'abatised', 'abed', 'aborted', 'abridged', 'abscessed', 'absconded', 'absorbed', 'abstracted', 'abstricted', 'accelerated', 'accepted', 'accidented', 'accoladed', 'accolated', 'accomplished', 'accosted', 'accredited', 'accursed', 'accused', 'accustomed', 'acetated', 'acheweed', 'aciculated', 'aciliated', 'acknowledged', 'acorned', 'acquainted', 'acquired', 'acquisited', 'acred', 'aculeated', 'addebted', 'added', 'addicted', 'addlebrained', 'addleheaded', 'addlepated', 'addorsed', 'adempted', 'adfected', 'adjoined', 'admired', 'admitted', 'adnexed', 'adopted', 'adossed', 'adreamed', 'adscripted', 'aduncated', 'advanced', 'advised', 'aeried', 'aethered', 'afeared', 'affected', 'affectioned', 'affined', 'afflicted', 'affricated', 'affrighted', 'affronted', 'aforenamed', 'afterfeed', 'aftershafted', 'afterthoughted', 'afterwitted', 'agazed', 'aged', 'agglomerated', 'aggrieved', 'agminated', 'agnamed', 'agonied', 'agreed', 'agueweed', 'ahungere

### Exercise Attendance:

Please search 100 words which finished "~ing" as progressive form in the wordlist on previous example.
進行形 "ing" で終わる単語を探して100件までリスト化してください。

['abhorring', 'abiding', 'abounding', 'absorbing', 'abutting', 'accommodating', 'according', 'accounting', 'aching', 'acting', 'adeling', 'adjoining', 'admiring', 'adsmithing', 'advancing', 'advertising', 'affecting', 'afflicting', 'affronting', 'afluking', 'afterburning', 'aftercoming', 'afterking', 'afterplanting', 'afterreckoning', 'afterripening', 'afterspring', 'afterswarming', 'afterworking', 'aggravating', 'aging', 'agoing', 'agreeing', 'ailing', 'aiming', 'airing', 'aisling', 'alarming', 'allthing', 'alluring', 'almsgiving', 'alternating', 'amazing', 'ambling', 'ambuling', 'amusing', 'anglewing', 'angling', 'animating', 'annoying', 'antespring', 'antiagglutinating', 'antiboxing', 'anticlogging', 'anticoagulating', 'anticovenanting', 'anticreeping', 'antidancing', 'antidetonating', 'antidumping', 'antiexporting', 'antiflattering', 'antifoaming', 'antifouling', 'antifreezing', 'antigambling', 'antiganting', 'antihunting', 'antiking', 'antileveling', 'antilynching', 'antimixing', 'antioxidizing', 'antipooling', 'antipriming', 'antiprofiteering', 'antiquing', 'antiracing', 'antiradiating', 'antirebating', 'antirecruiting', 'antireforming', 'antishipping', 'antiskidding', 'antismoking', 'antisplitting', 'antispreading', 'antisquatting', 'antistalling', 'antivibrating', 'antling', 'anubing', 'anything', 'apeling', 'aping', 'appalling', 'appealing', 'appeasing', 'appraising', 'approaching']

'.' はワイルドカード記号。下の例は、8文字、3番目にj, 6番目にtが来る文字列を抽出。

In [ ]:
 print([w for w in wordlist if re.search('^..j..t..$', w)])

['abjectly', 'adjuster', 'dejected', 'dejectly', 'injector', 'majestic', 'objectee', 'objector', 'rejecter', 'rejector', 'unjilted', 'unjolted', 'unjustly']


RangesとClosures
![image.png](attachment:image.png)
 - T9システム: スマホに利用される。
 - キーストロークの同じシーケンスで入力された2つ以上の単語を、textonymsという。それを見つけられるか？

In [ ]:
print([w for w in wordlist if re.search('^[ghi][mno][jlk][def]$', w)])

['gold', 'golf', 'hold', 'hole']


Kleene Closure (Closure): '+'や'\*' 記号で代表される、特定の記号列が連続して出現する生成パターンにマッチングするケース。

In [ ]:
chat_words = sorted(set(w for w in nltk.corpus.nps_chat.words()))
print([w for w in chat_words if re.search('^m+i+n+e+$', w)])

['miiiiiiiiiiiiinnnnnnnnnnneeeeeeeeee', 'miiiiiinnnnnnnnnneeeeeeee', 'mine', 'mmmmmmmmiiiiiiiiinnnnnnnnneeeeeeee']


Class: '[',']' に囲まれた中の文字を1つの文字の候補として纏める。

In [ ]:
print([w for w in chat_words if re.search('^[ha]+$', w)])

['a', 'aaaaaaaaaaaaaaaaa', 'aaahhhh', 'ah', 'ahah', 'ahahah', 'ahh', 'ahhahahaha', 'ahhh', 'ahhhh', 'ahhhhhh', 'ahhhhhhhhhhhhhh', 'h', 'ha', 'haaa', 'hah', 'haha', 'hahaaa', 'hahah', 'hahaha', 'hahahaa', 'hahahah', 'hahahaha', 'hahahahaaa', 'hahahahahaha', 'hahahahahahaha', 'hahahahahahahahahahahahahahahaha', 'hahahhahah', 'hahhahahaha']


In [ ]:
wsj = sorted(set(nltk.corpus.treebank.words()))
print([w for w in wsj if re.search('^[0-9]+\.[0-9]+$', w)][:100])

['0.0085', '0.05', '0.1', '0.16', '0.2', '0.25', '0.28', '0.3', '0.4', '0.5', '0.50', '0.54', '0.56', '0.60', '0.7', '0.82', '0.84', '0.9', '0.95', '0.99', '1.01', '1.1', '1.125', '1.14', '1.1650', '1.17', '1.18', '1.19', '1.2', '1.20', '1.24', '1.25', '1.26', '1.28', '1.35', '1.39', '1.4', '1.457', '1.46', '1.49', '1.5', '1.50', '1.55', '1.56', '1.5755', '1.5805', '1.6', '1.61', '1.637', '1.64', '1.65', '1.7', '1.75', '1.76', '1.8', '1.82', '1.8415', '1.85', '1.8500', '1.9', '1.916', '1.92', '10.19', '10.2', '10.5', '107.03', '107.9', '109.73', '11.10', '11.5', '11.57', '11.6', '11.72', '11.95', '112.9', '113.2', '116.3', '116.4', '116.7', '116.9', '118.6', '12.09', '12.5', '12.52', '12.68', '12.7', '12.82', '12.97', '120.7', '1206.26', '121.6', '126.1', '126.15', '127.03', '129.91', '13.1', '13.15', '13.5', '13.50', '13.625']


In [ ]:
print([w for w in wsj if re.search('^[A-Z]+\$$', w)])

['C$', 'US$']


In [ ]:
print([w for w in wsj if re.search('^[0-9]{4}$', w)])

['1614', '1637', '1787', '1901', '1903', '1917', '1925', '1929', '1933', '1934', '1948', '1953', '1955', '1956', '1961', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1975', '1976', '1977', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2005', '2009', '2017', '2019', '2029', '3057', '8300']


In [ ]:
print([w for w in wsj if re.search('^[0-9]+-[a-z]{3,5}$', w)])

['10-day', '10-lap', '10-year', '100-share', '12-point', '12-year', '14-hour', '15-day', '150-point', '190-point', '20-point', '20-stock', '21-month', '237-seat', '240-page', '27-year', '30-day', '30-point', '30-share', '30-year', '300-day', '36-day', '36-store', '42-year', '50-state', '500-stock', '52-week', '69-point', '84-month', '87-store', '90-day']


In [ ]:
print([w for w in wsj if re.search('^[a-z]{5,}-[a-z]{2,3}-[a-z]{,6}$', w)])

['black-and-white', 'bread-and-butter', 'father-in-law', 'machine-gun-toting', 'savings-and-loan']


In [ ]:
print([w for w in wsj if re.search('(ed|ing)$', w)][:100])

['62%-owned', 'Absorbed', 'According', 'Adopting', 'Advanced', 'Advancing', 'Alfred', 'Allied', 'Annualized', 'Anything', 'Arbitrage-related', 'Arbitraging', 'Asked', 'Assuming', 'Atlanta-based', 'Baking', 'Banking', 'Beginning', 'Beijing', 'Being', 'Bermuda-based', 'Betting', 'Boeing', 'Broadcasting', 'Bucking', 'Buying', 'Calif.-based', 'Change-ringing', 'Citing', 'Concerned', 'Confronted', 'Conn.based', 'Consolidated', 'Continued', 'Continuing', 'Declining', 'Defending', 'Depending', 'Designated', 'Determining', 'Developed', 'Died', 'During', 'Encouraged', 'Encouraging', 'English-speaking', 'Estimated', 'Everything', 'Excluding', 'Exxon-owned', 'Faulding', 'Fed', 'Feeding', 'Filling', 'Filmed', 'Financing', 'Following', 'Founded', 'Fracturing', 'Francisco-based', 'Fred', 'Funded', 'Funding', 'Generalized', 'Germany-based', 'Getting', 'Guaranteed', 'Having', 'Heating', 'Heightened', 'Holding', 'Housing', 'Illuminating', 'Indeed', 'Indexing', 'Irving', 'Jersey-based', 'Judging', 'Know

![image.png](attachment:image.png)
上図: ワイルドカード、範囲、閉包を含む基本的な正規表現のメタキャラクタ

## Lesson 4. Useful Application of Regular Expression (3.5.)

### 4.1. Extracting subsequence of a word

re.findall()メソッドは、マッチングするすべての表現を見つける機能を持つ。これを用いて、Word Piecesの抽出を行う

In [ ]:
word = 'supercalifragilisticexpialidocious'
print(re.findall(r'[aeiou]', word))
print(len(re.findall(r'[aeiou]', word)))

['u', 'e', 'a', 'i', 'a', 'i', 'i', 'i', 'e', 'i', 'a', 'i', 'o', 'i', 'o', 'u']
16


任意のテキスト内の2つ以上の連続した母音を全て調べ、それらの相対頻度を調べる。

In [ ]:
wsj = sorted(set(nltk.corpus.treebank.words()))
fd = nltk.FreqDist(vs for word in wsj
     for vs in re.findall(r'[aeiou]{2,}', word))
print(fd.most_common(12))

[('io', 549), ('ea', 476), ('ie', 331), ('ou', 329), ('ai', 261), ('ia', 253), ('ee', 217), ('oo', 174), ('ua', 109), ('au', 106), ('ue', 105), ('ui', 95)]


### 4.2. Doing more with word pieces

 - 母音を抜いて、子音のみを集めることでも、英語は意味を簡単に理解できる。その操作を行う例

In [ ]:
regexp = r'^[AEIOUaeiou]+|[AEIOUaeiou]+$|[^AEIOUaeiou]'
def compress(word):
    pieces = re.findall(regexp, word)
    return ''.join(pieces)
english_udhr = nltk.corpus.udhr.words('English-Latin1')
print(nltk.tokenwrap(english_udhr[:75]))
print(nltk.tokenwrap(compress(w) for w in english_udhr[:75]))

Universal Declaration of Human Rights Preamble Whereas recognition of
the inherent dignity and of the equal and inalienable rights of all
members of the human family is the foundation of freedom , justice and
peace in the world , Whereas disregard and contempt for human rights
have resulted in barbarous acts which have outraged the conscience of
mankind , and the advent of a world in which human beings shall enjoy
freedom of speech and
Unvrsl Dclrtn of Hmn Rghts Prmble Whrs rcgntn of the inhrnt dgnty and
of the eql and inlnble rghts of all mmbrs of the hmn fmly is the fndtn
of frdm , jstce and pce in the wrld , Whrs dsrgrd and cntmpt fr hmn
rghts hve rsltd in brbrs acts whch hve outrgd the cnscnce of mnknd ,
and the advnt of a wrld in whch hmn bngs shll enjy frdm of spch and


正規表現を条件付き頻度分布との組み合わせで扱う: ロカトス語の単語から抜き出した全ての子音-母音の組み合わせ表。

In [ ]:
rotokas_words = nltk.corpus.toolbox.words('rotokas.dic')
cvs = [cv for w in rotokas_words for cv in re.findall(r'[ptksvr][aeiou]', w)]
cfd = nltk.ConditionalFreqDist(cvs)
print(cfd.tabulate())

    a   e   i   o   u 
k 418 148  94 420 173 
p  83  31 105  34  51 
r 187  63  84  89  79 
s   0   0 100   2   1 
t  47   8   0 148  37 
v  93  27 105  48  49 
None


索引 (indexing): 'su'を含むすべての単語を返す、など。

In [ ]:
cv_word_pairs = [(cv, w) for w in rotokas_words
                         for cv in re.findall(r'[ptksvr][aeiou]', w)]
cv_index = nltk.Index(cv_word_pairs)
print("'su': ", cv_index['su'])
print("'po': ", cv_index['po'])

'su':  ['kasuari']
'po':  ['kaapo', 'kaapopato', 'kaipori', 'kaiporipie', 'kaiporivira', 'kapo', 'kapoa', 'kapokao', 'kapokapo', 'kapokapo', 'kapokapoa', 'kapokapoa', 'kapokapora', 'kapokapora', 'kapokaporo', 'kapokaporo', 'kapokari', 'kapokarito', 'kapokoa', 'kapoo', 'kapooto', 'kapoovira', 'kapopaa', 'kaporo', 'kaporo', 'kaporopa', 'kaporoto', 'kapoto', 'karokaropo', 'karopo', 'kepo', 'kepoi', 'keposi', 'kepoto']


### 4.3. Find trunk of word

Word Stemの検索: ing, ly, ed, ious, ies, ive, es, s, ment等の接尾語を取り除く場合
 - 最終的にはNLTK組み込みステマーを使用するが、この場所では正規表現で行ってみる。

In [ ]:
def stem(word):
     for suffix in ['ing', 'ly', 'ed', 'ious', 'ies', 'ive', 'es', 's', 'ment']:
        if word.endswith(suffix):
            return word[:-len(suffix)]
        return word

正規表現は全体にマッチするが、取得できるのは接尾語のみ

In [ ]:
print(re.findall(r'^.*(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing'))

['ing']


?を追加して出力するマテリアルの選択

In [ ]:
print(re.findall(r'^.*(?:ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing'))

['processing']


Stemと接尾語に分けるために、正規表現の両方に括弧で囲む。

In [ ]:
print(re.findall(r'^(.*)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing'))

[('process', 'ing')]


*がGreedyの一例: 誤ってsが出てくる

In [ ]:
print(re.findall(r'^(.*)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processes'))

[('processe', 's')]


貪欲でないVersion(*?)を利用すると、望むものを得られる。

In [ ]:
print(re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processes'))

[('process', 'es')]


空の接尾語を許可して2番目の括弧の内容をオプションにすることで機能。

In [ ]:
print(re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$', 'language'))

[('language', '')]


関数化し、テキスト全体に適用。

In [ ]:
def stem(word):
    regexp = r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$'
    stem, suffix = re.findall(regexp, word)[0]
    return stem
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
is no basis for a system of government.  Supreme executive power derives from
a mandate from the masses, not from some farcical aquatic ceremony."""

tokens = word_tokenize(raw)
print([stem(t) for t in tokens])

['DENNIS', ':', 'Listen', ',', 'strange', 'women', 'ly', 'in', 'pond', 'distribut', 'sword', 'i', 'no', 'basi', 'for', 'a', 'system', 'of', 'govern', '.', 'Supreme', 'execut', 'power', 'deriv', 'from', 'a', 'mandate', 'from', 'the', 'mass', ',', 'not', 'from', 'some', 'farcical', 'aquatic', 'ceremony', '.']


### 4.4. トークン化テキストの検索
特殊な種類の正規表現を利用してテキスト内の複数の単語を検索する。テキスト内の男性の全てのインスタンスを検索。

In [ ]:
from nltk.corpus import gutenberg, nps_chat
moby = nltk.Text(gutenberg.words('melville-moby_dick.txt'))
print(moby.findall(r"<a> (<.*>) <man>"))

monied; nervous; dangerous; white; white; white; pious; queer; good;
mature; white; Cape; great; wise; wise; butterless; white; fiendish;
pale; furious; better; certain; complete; dismasted; younger; brave;
brave; brave; brave
None


In [ ]:
chat = nltk.Text(nps_chat.words()) #単語broで終わる3語の句を探す
print(chat.findall(r"<.*> <.*> <bro>") )

you rule bro; telling you bro; u twizted bro
None


In [ ]:
print(chat.findall(r"<l.*>{3,}")) # ｌで始まる単語が3語以上連続している部分を探す。

lol lol lol; lmao lol lol; lol lol lol; la la la la la; la la la; la
la la; lovely lol lol love; lol lol lol.; la la la; la la la
None


特定の単語を対象とした言語現象を研究対象とする場合、検索パターンを作るのは比較的容易。  
例えば、x and other ys という正規表現で大きなテキストコーパスを検索することで、上位語を発見することが出来る。

In [ ]:
from nltk.corpus import brown
hobbies_learned = nltk.Text(brown.words(categories=['hobbies', 'learned']))
hobbies_learned.findall(r"<\w*> <and> <other> <\w*s>")

speed and other activities; water and other liquids; tomb and other
landmarks; Statues and other monuments; pearls and other jewels;
charts and other items; roads and other features; figures and other
objects; military and other areas; demands and other factors;
abstracts and other compilations; iron and other metals


## Lesson 5. テキストの正規化 (3.6.)

正規化:
 - lower() : 小文字に『正規化』する
 - stemming: 単語から全ての接辞（接頭・接尾語等）を取り除く。
 - lemmatize 見出し語化: 語の形を辞書に記述されている語形に変換する作業。
 
以下のデータを利用する。

In [ ]:
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
is no basis for a system of government.  Supreme executive power derives from
a mandate from the masses, not from some farcical aquatic ceremony."""
tokens = word_tokenize(raw)

NameError: ignored

### 5.1. Stemmers

NLTKに含まれるStemmerを利用
 - PorterStemmer() 情報検索のために作られた古典的ステマー
 - LancasterStemmer() 英語用の古典的ステマー

In [ ]:
porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()
print([porter.stem(t) for t in tokens])

NameError: ignored

ステマーを用いたテキストのインデックス構築

In [ ]:
class IndexedText(object):

    def __init__(self, stemmer, text):
        self._text = text
        self._stemmer = stemmer
        self._index = nltk.Index((self._stem(word), i)
                                 for (i, word) in enumerate(text))

    def concordance(self, word, width=40):
        key = self._stem(word)
        wc = int(width/4)                # words of context
        for i in self._index[key]:
            lcontext = ' '.join(self._text[i-wc:i])
            rcontext = ' '.join(self._text[i:i+wc])
            ldisplay = '{:>{width}}'.format(lcontext[-width:], width=width)
            rdisplay = '{:{width}}'.format(rcontext[:width], width=width)
            print(ldisplay, rdisplay)

    def _stem(self, word):
        return self._stemmer.stem(word).lower()

In [ ]:
porter = nltk.PorterStemmer()
grail = nltk.corpus.webtext.words('grail.txt')
text = IndexedText(porter, grail)
text.concordance('lie')

r king ! DENNIS : Listen , strange women lying in ponds distributing swords is no
 beat a very brave retreat . ROBIN : All lies ! MINSTREL : [ singing ] Bravest of
       Nay . Nay . Come . Come . You may lie here . Oh , but you are wounded !   
doctors immediately ! No , no , please ! Lie down . [ clap clap ] PIGLET : Well  
ere is much danger , for beyond the cave lies the Gorge of Eternal Peril , which 
   you . Oh ... TIM : To the north there lies a cave -- the cave of Caerbannog --
h it and lived ! Bones of full fifty men lie strewn about its lair . So , brave k
not stop our fight ' til each one of you lies dead , and the Holy Grail returns t


### 5.2. Lemmatization
見出し語化: Wordnetの字句解析ツールは、結果の単語が辞書に含まれている場合にのみ、接尾語を削除します。

In [ ]:
wnl = nltk.WordNetLemmatizer()
print([wnl.lemmatize(t) for t in tokens])

['DENNIS', ':', 'Listen', ',', 'strange', 'woman', 'lying', 'in', 'pond', 'distributing', 'sword', 'is', 'no', 'basis', 'for', 'a', 'system', 'of', 'government', '.', 'Supreme', 'executive', 'power', 'derives', 'from', 'a', 'mandate', 'from', 'the', 'mass', ',', 'not', 'from', 'some', 'farcical', 'aquatic', 'ceremony', '.']


## Lesson 6. Formatting: From Lists to String (3.9.)

### 6.1. リストから文字列
リストから文字列に変換すると、より良い出力が出来る可能性がある。

In [ ]:
silly = ['We', 'called', 'him', 'Tortoise', 'because', 'he', 'taught', 'us', '.']
print('blank: ', ' '.join(silly))
print('semicolon: ', ';'.join(silly))
print('tight: ', ''.join(silly))

blank:  We called him Tortoise because he taught us .
semicolon:  We;called;him;Tortoise;because;he;taught;us;.
tight:  WecalledhimTortoisebecausehetaughtus.


### 6.2. 文字列と整形

print文: Pythonがオブジェクトの内容を人間に見やすいように整形する方法。  
プロンプトで変数の名前を直接指定する方法←簡単だが、積極的には出力されない。  
どちらも単なる文字列。

In [ ]:
word = 'cat'
sentence = """hello
world"""
print(word)
print(sentence)

cat
hello
world


In [ ]:
word

'cat'

In [ ]:
sentence

'hello\nworld'

プリント文を使ってフォーマッティングをすると面倒くさい。

In [ ]:
fdist = nltk.FreqDist(['dog', 'cat', 'dog', 'cat', 'dog', 'snake', 'dog', 'cat'])
for word in sorted(fdist):
    print(word, '->', fdist[word], end='; ')

cat -> 3; dog -> 4; snake -> 1; 

文字列整形式を利用すると非常に使いやすい。

In [ ]:
for word in sorted(fdist):
   print('{}->{};'.format(word, fdist[word]), end=' ')

cat->3; dog->4; snake->1; 

In [ ]:
print('{}->{};'.format ('cat', 3))

cat->3;


In [ ]:
print('{}->'.format('cat'))

cat->


In [ ]:
print('{}'.format(3))

3


In [ ]:
print('I want a {} right now'.format('coffee'))

I want a coffee right now


In [ ]:
print('{} wants a {} {}'.format ('Lee', 'sandwich', 'for lunch'))

Lee wants a sandwich for lunch


'{} wants a {} {}'.format ('sandwich', 'for lunch')

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
<ipython-input-92-339b7c04a9c2> in <module>
----> 1 '{} wants a {} {}'.format ('sandwich', 'for lunch')

IndexError: tuple index out of range

In [ ]:
print('{} wants a {}'.format ('Lee', 'sandwich', 'for lunch'))

Lee wants a sandwich


In [ ]:
print('from {1} to {0}'.format('A', 'B'))

from B to A


In [ ]:
template = 'Lee wants a {} right now'
menu = ['sandwich', 'spam fritter', 'pancake']
for snack in menu:
    print(template.format(snack))

Lee wants a sandwich right now
Lee wants a spam fritter right now
Lee wants a pancake right now


### 6.3. 値を並べる

Lining things Up: パディングして出力

In [ ]:
print('{:6}'.format(41))

    41


In [ ]:
print('{:<6}' .format(41))

41    


In [ ]:
print('{:6}'.format('dog'))

dog   


In [ ]:
print('{:>6}'.format('dog') )

   dog


In [ ]:
import math
print('{:.4f}'.format(math.pi))

3.1416


In [ ]:
count, total = 3205, 9375
print("accuracy for {} words: {:.4%}".format(total, count / (1.0* total)))

accuracy for 9375 words: 34.1867%


In [ ]:
def tabulate(cfdist, words, categories):
    print('{:16}'.format('Category'), end=' ')                    # column headings
    for word in words:
        print('{:>6}'.format(word), end=' ')
    print()
    for category in categories:
        print('{:16}'.format(category), end=' ')                  # row heading
        for word in words:                                        # for each word
            print('{:6}'.format(cfdist[category][word]), end=' ') # print table cell
        print()                                                   # end the row

In [ ]:
from nltk.corpus import brown
cfd = nltk.ConditionalFreqDist(
          (genre, word)
          for genre in brown.categories()
          for word in brown.words(categories=genre))
genres = ['news', 'religion', 'hobbies', 'science_fiction', 'romance', 'humor']
modals = ['can', 'could', 'may', 'might', 'must', 'will']
tabulate(cfd, modals, genres)

Category            can  could    may  might   must   will 
news                 93     86     66     38     50    389 
religion             82     59     78     12     54     71 
hobbies             268     58    131     22     83    264 
science_fiction      16     49      4     12      8     16 
romance              74    193     11     51     45     43 
humor                16     30      8      8      9     13 


In [ ]:
print('{:{width}}'.format('Monty Python', width=15))

Monty Python   


### 6.4. ファイル出力

In [ ]:
output_file = open('output.txt', 'w')
words = set(nltk.corpus.genesis.words('english-kjv.txt'))
for word in sorted(words):
    print(word, file=output_file)

In [ ]:
len(words)

2789

In [ ]:
str(len(words))

'2789'

In [ ]:
print(str(len(words)), file=output_file)
output_file.close()

### 6.5. Text Wrapping: %s, %d等

In [ ]:
saying = ['After', 'all', 'is', 'said', 'and', 'done', ',',
         'more', 'is', 'said', 'than', 'done', '.']
for word in saying:
    print(word, '(' + str(len(word)) + '),', end=' ')

After (5), all (3), is (2), said (4), and (3), done (4), , (1), more (4), is (2), said (4), than (4), done (4), . (1), 

In [ ]:
from textwrap import fill
format = '%s (%d),'
pieces = [format % (word, len(word)) for word in saying]
output = ' '.join(pieces)
wrapped = fill(output)
print(wrapped)

After (5), all (3), is (2), said (4), and (3), done (4), , (1), more
(4), is (2), said (4), than (4), done (4), . (1),
